In [1]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import time
import shutil
import tqdm
import pickle
import tflearn
import numpy as np
from sklearn.cross_validation import train_test_split
from random import shuffle
from keras.preprocessing.image import ImageDataGenerator


from keras.callbacks import TensorBoard
from keras import backend as K
from keras.layers import Conv2D, Embedding, Dense, LSTM, TimeDistributed, concatenate, Input, SimpleRNN, Dropout, Activation
from keras.models import Model, Sequential
from keras.applications import InceptionV3 

%matplotlib inline

Using TensorFlow backend.
c:\users\vinayak tyagi\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# setting up question and importing up the file

# Question = 'What are they doing ?'

# jog_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jogging-20180627T014223Z-001\\jogging'
# walk_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\Walking-20180627T014406Z-001\\Walking'
# tt_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\train-testing-data'


parent_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk'
test_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test'
train_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\tain'
valid_dir = 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\valid'

In [3]:
# creating the labels of the data
def label(vid):
    word_label = vid.split('_')[-3]
    if word_label == 'jogging':  return[1,0]
    elif word_label == 'walking': return[0,1]

In [4]:
def create_train_data():
    train_data = []
    for vid in os.listdir(train_dir):
        labels = label(vid)
        path = os.path.join(train_dir,vid)
        vid = cv2.VideoCapture(path)
        train_data.append([np.array(vid), np.array(labels)])
    shuffle(train_data)
#     np.save('jogVswalk.npy',train_data)
#     print(train_data)
    return train_data
        

In [5]:
list_of_folder = []
def make_dir():
    a = os.listdir(parent_dir)

    for i in range(len(a)):
        child_dir = parent_dir + "\\"+str(a[i])
        m = os.listdir(child_dir)

        for j in range(len(m)):
                kid_dir = child_dir + "\\"+str(m[j])
                z = os.listdir(kid_dir)
    #             print('\n---------------******************--------------------\n')
    #             print(z)

                for p in range(len(z)):
                    infant_dir = kid_dir + "\\folder-" +str(p)
                    list_of_folder.append(infant_dir)
#                     print(infant_dir)
        
                    if not os.path.isdir(infant_dir):
                        os.makedirs(infant_dir) 

                        cap = cv2.VideoCapture(kid_dir +"\\"+ z[p])
                        count = 0

                        while cap.isOpened():
                            ret,frame = cap.read()
#                             print(ret)
                            if ret == True:
                                cv2.imshow('window-name',frame)
                                cv2.imwrite(infant_dir+"\\ frame-%d.jpg" % count, frame)
                                count = count + 1
#                                 print(count)
                                if cv2.waitKey(10) & 0xFF == ord('q'):
                                    break
                            else: 
                                    break

#                         print(count)
                        cap.release()
                        cv2.destroyAllWindows()
           

#  Now it's time to process the list file            
if not os.path.isfile("C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test.txt"):
    make_dir()

if os.path.isfile("C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test.txt"):
    with open("C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test.txt","rb") as fp:
        list_of_folder =  pickle.load(fp)
        if len(list_of_folder) == 0:
            make_dir()

if len(list_of_folder)  == 98:
    print(list_of_folder)
    with open("C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test.txt","wb") as fp:
        pickle.dump(list_of_folder , fp)
        print("File Build Sucessfully")
        

if len(list_of_folder) < 98:
    for i in range(len(list_of_folder)):
        shutil.rmtree(list_of_folder[i])
    print("Data removed Sucessfully")
        
    
# if a[0] == 'jog':
#     open_path = test_dir + "\\jog"
    
#     no_vid = os.listdir(open_path)
#     print(no_vid)
#     for i in range(len(no_vid)):
#         folder = open_path + "\\ folder-"+str(i)
#         if not os.path.isdir(folder):
#             os.makedirs(folder)
             
#             cap = cv2.VideoCapture(open_path +"\\"+ no_vid[i])
#             count = 0

#             while cap.isOpened():
#                 ret,frame = cap.read()
#                 if ret == True:
#                     cv2.imshow('window-name',frame)
#                     cv2.imwrite(folder+"\\ frame-%d.jpg" % count, frame)
#                     count = count + 1
#                     print(count)
#                     if cv2.waitKey(10) & 0xFF == ord('q'):
#                         break
#                 else: 
#                     break

#             print(count)
#             cap.release()
#             cv2.destroyAllWindows()
       

    
# data = create_train_data()
# data

['C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-0', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-1', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-2', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-3', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-4', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-5', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-6', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-7', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-8', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-9', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-10', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\walk\\folder-0', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\walk\\folder

In [6]:
test_data =[]
for i in range(len(list_of_folder)):
#     print(list_of_folder[i])
    a = list_of_folder[i].split("\\")
    test_data.append(a[-2])

train_x, test_x, train_y, test_y = train_test_split(list_of_folder, test_data,  test_size = 0.3)

print(train_x)
print(train_y)

['C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\walk\\folder-20', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\walk\\folder-30', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\jog\\folder-27', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\walk\\folder-12', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-10', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\jog\\folder-8', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\walk\\folder-8', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\test\\jog\\folder-9', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\valid\\jog\\folder-0', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\valid\\walk\\folder-3', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\jog\\folder-10', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\\train\\walk\\folder-4', 'C:\\Users\\vinayak tyagi\\Downloads\\Rough\\jog&walk\

In [ ]:
keras_model = Sequential()
keras_model.add(Conv3D(8, (5, 5, 5), input_shape=(3, 8, 8, 8), name='conv'))
keras_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD())

In [8]:
# train , test = train_test_split(data, test_size = 0.3, random_state = 7)

In [9]:
# data = np.array(data)
# data.shape

# def data_genrator(given):
#     a = []
#     b = []

#     for i in range(len(given)):
#         a.append(given[i][0].tolist())
#         b.append(given[i][1].tolist())
#     return a,b

# x_train, y_train = data_genrator(train) 
# x_test, y_test = data_genrator(test)

In [10]:
# print(len(x_train)); print(len(x_test))

In [11]:
# x_train = np.array(x_train)
# a = x_train.reshape(x_train.shape[0], 1, 1)
# print(a)
# print(y_train)

In [12]:
# # # now making the model

# vedio = Input(shape=(None,213,213,3))
# cnn = InceptionV3(weights='imagenet',
#                  include_top=False)

# cnn.trainable = False
# encoded_frames = TimeDistributed(cnn)(vedio)
# print(encoded_frames)
# encoded_vid = LSTM(512, return_sequences=False)(encoded_frames)


# question = Input(shape=(100), dtype='int32')
# x = Embeeding(10000,256, mask_zero=True)(question) 
# encoded_q = LSTM(128)(x)

# x = concatenate([encoded_vid, encoded_q])
# x = Dense(128,activation=tf.nn.relu)(x)
# outputs = Dense(10000)(x)

# model = Model([video,question],outputs)

# model.compile(optimizer = tf.AdamOptimizer(),
# #              loss = tf.softmax_crossentropy_with_logits)

# model = Sequential()

# model.add(LSTM(150,
#                input_shape=( 7, 339),
#                return_sequences=True))
# model.add(Dropout(0.2))

# model.add(LSTM(
#     200,
#     return_sequences=True))
# model.add(Dropout(0.2))

# model.add(LSTM(
#     150,
#     return_sequences=True))
# model.add(Dropout(0.2))

# model.add(Dense(
#     output_dim=1))
# model.add(Activation('sigmoid'))

# start = time.time()

# model.compile(loss='mse', optimizer='rmsprop')

# print('compilation time : ', time.time() - start)

# model.fit(x_train, y_train, batch_size=64, nb_epoch=10)
